In [ ]:
import pyspiel
import open_spiel
import pandas as pd
import numpy as np
fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})

leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
import copy
class WrapperEnv:
    def __init__(self,game):
        self.game= game
        self.state = game.new_initial_state()
        self.agent_selection = str(self.state.current_player())
        self.traverser = None
    
    def reset(self, seed=None):
        self.state = self.game.new_initial_state()
        while self.state.is_chance_node():
            self.state.apply_action(np.random.choice(self.state.legal_actions()))
        self.agent_selection =  str(self.state.current_player())
        return self.obs()
    
    def step(self, action):
        if self.state.is_chance_node():
            while self.state.is_chance_node():
                self.state.apply_action(np.random.choice(self.state.legal_actions()))

        else:
            self.state.apply_action(action)
            if self.state.is_chance_node():
                while self.state.is_chance_node():
                    self.state.apply_action(np.random.choice(self.state.legal_actions()))
        
        if self.state.is_terminal():
            return self.obs()
        else:
            # store = copy.deepcopy(self.state)
            while self.state.is_chance_node():
                self.state.apply_action(np.random.choice(self.state.legal_actions()))
            # print("3")
            # print(self.state.is_terminal())
            # if self.state.is_terminal():
            #     print("store:", store)
            # print(self.state)
            # print(self.state.legal_actions_mask(int(self.agent_selection)))
            # print("3")
            self.agent_selection =  str(self.state.current_player())

            return self.obs()
    
    def last(self):
        return self.obs()
    
    def obs(self):
        return {"observation":self.state.observation_tensor(int(self.agent_selection)), "action_mask":np.stack(self.state.legal_actions_mask(int(self.agent_selection)))}, self.state.player_reward(self.traverser) if self.traverser is not None else self.state.player_return(int(self.agent_selection)), self.state.is_terminal(), False, "OPENSPIEL"

leducgame = WrapperEnv(leduc)
fhpgame = WrapperEnv(fhp)


/var/folders/q_/hxdspxw938n3_89qddxh_2fm0000gp/T/ipykernel_69048/662883081.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
leducgame.reset()

({'observation': [1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   100.0,
   100.0],
  'action_mask': array([0, 1, 1, 1])},
 0.0,
 False,
 False,
 'OPENSPIEL')

In [ ]:
from pettingzoo.classic import texas_holdem_v4
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
import torch
from cfr_network import CFRNetwork
games = [leducgame, fhpgame]
hidden_dim = 128
input_dim = 108
output_dim = 4
num_players = 2
replay_buffer_size = 4000000
minibatch_size = 10000
steps_per_epoch = 3000
traversals = 3000
training_steps = 20000
lr = 0.0001
optimizer = None
p_v_networks = {'input_shape':input_dim, 'output_shape':output_dim, 'hidden_size':hidden_dim, 'learning_rate':lr, 'optimizer':optimizer}
active_player_obj = ActivePlayer(num_players)
config = CFRConfig(
    config_dict={'network': {'policy': p_v_networks, 'value': p_v_networks, 'num_players':num_players},
                 'replay_buffer_size':replay_buffer_size,
                 'minibatch_size':minibatch_size,
                 'steps_per_epoch':steps_per_epoch,
                 'traversals': traversals,
                 'training_steps': training_steps,
                 'active_player_obj': active_player_obj,
                 },
    game_config={'num_players':num_players,
                 'observation_space':108,
                 'action_space':4,},
)
sampling = ["Full","MC"]

for i in sampling:
    for o in games:
        if o == fhpgame:
            config.game_config['observation_space'] = 108
            config.game_config['action_space'] = 4
            modelname = "CFR_FHP_" + i
        elif o == leducgame:
            config.game_config['observation_space'] = 16
            config.game_config['action_space'] = 4
            modelname = "CFR_LEDUC_" + i
        model = CFRAgent(env=o,config=config, name="CFR" +  + i, max_nodes=10000000)
        model.train(sampling=i)

/Users/Ile-Maurice/Library/Python/3.10/lib/python/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


CFRConfig
Iteration 0 done
Nodes touched 19038
Iteration 1 done
Nodes touched 51718
Iteration 2 done
Nodes touched 69718
Iteration 3 done
Nodes touched 87718
Iteration 4 done
Nodes touched 105718
Iteration 5 done
Nodes touched 123718
Iteration 6 done
Nodes touched 141718
Iteration 7 done
Nodes touched 159718
Iteration 8 done
Nodes touched 177718
Iteration 9 done
Nodes touched 195718
Iteration 10 done
Nodes touched 213718
Iteration 11 done
Nodes touched 231718
Iteration 12 done
Nodes touched 249718
Iteration 13 done
Nodes touched 267718
Iteration 14 done
Nodes touched 285718
Iteration 15 done
Nodes touched 303718
Iteration 16 done
Nodes touched 321718
Iteration 17 done
Nodes touched 339718
Iteration 18 done
Nodes touched 357718
Iteration 19 done
Nodes touched 375718
Iteration 20 done
Nodes touched 397762
Iteration 21 done
Nodes touched 427756
Iteration 22 done
Nodes touched 445756
Iteration 23 done
Nodes touched 463756
Iteration 24 done
Nodes touched 481756
Iteration 25 done
Nodes touch